In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import seaborn as sns
from census import Census
import statsmodels.formula.api as smf
import statsmodels.api as sm
import scipy.stats as sps
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, ElasticNetCV,LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error,accuracy_score,precision_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.impute import SimpleImputer
import sklearn
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
def metric_count(y,y_hat):
    acc = accuracy_score(y,y_hat)
    precision = precision_score(y,y_hat, average='macro')
    recall = recall_score(y,y_hat, average='macro')
    f1 = f1_score(y,y_hat, average='macro')

    return print('acc:',acc,'pre:',precision,'re',recall, 'f1',f1)

In [11]:
df = pd.read_csv('preprocessed_data.csv')
df

,Unnamed: 0,SHRP_ID,STATE_CODE,VISIT_DATE,IRI_LEFT_WHEEL_PATH,IRI_RIGHT_WHEEL_PATH,AADT_ALL_VEHIC,MEPDG_TRANS_CRACK_LENGTH_AC,Mean_IRI,IRI_condition,year,construction_time
0,0,0001,48,1990-03-06,0.6904,0.6570,-1.000000,12.881818,0.6737,2.0,1990,32
1,1,0001,48,1992-03-17,0.6894,0.7312,-1.000000,12.881818,0.7103,2.0,1992,30
2,2,0001,48,1993-07-13,0.7472,0.6938,-1.000000,12.881818,0.7205,2.0,1993,29
3,3,0001,48,1995-05-03,0.7092,0.7838,-1.000000,12.881818,0.7465,2.0,1995,27
4,4,0001,48,1997-08-06,0.7600,0.7508,-1.000000,12.881818,0.7554,2.0,1997,25
...,...,...,...,...,...,...,...,...,...,...,...,...
29765,29765,Q350,48,1995-05-01,1.2042,1.1674,1158.809524,-1.000000,1.1858,2.0,1995,27
29766,29766,Q350,48,2001-04-18,1.1592,1.0578,1158.809524,-1.000000,1.1085,2.0,2001,21
29767,29767,Q353,48,1990-09-17,1.1030,0.7784,1158.809524,2.183333,0.9407,2.0,1990,32
29768,29768,Q353,48,1995-05-01,1.2294,1.0444,1158.809524,2.183333,1.1369,2.0,1995,27


In [5]:
train, test = train_test_split(df, test_size=0.25, random_state=12345)

In [15]:
feat_cols = ['AADT_ALL_VEHIC','construction_time', 'MEPDG_TRANS_CRACK_LENGTH_AC']
out_col = 'Mean_IRI'
train_x = train[feat_cols]
train_y = train[out_col]
test_x = test[feat_cols]
test_y = test[out_col]

### regression

1. Random Forest

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import time

rm_pipe = Pipeline([
    ('columns', ColumnTransformer([
        ('AADT',StandardScaler(),['AADT_ALL_VEHIC']),
        ('Crack',StandardScaler(),['MEPDG_TRANS_CRACK_LENGTH_AC'])
    ])),
    ('model', RandomForestRegressor(max_depth=5, random_state=0,n_estimators=100))
])
start_time = time.time()
rm_pipe.fit(train_x,train_y)
end_time = time.time()
print('training-time:',end_time - start_time)
print('train-MSE:',mean_squared_error(train_y,rm_pipe.predict(train_x)))
print('train-r2:',r2_score(train_y,rm_pipe.predict(train_x)))
print('test-MSE:',mean_squared_error(test_y,rm_pipe.predict(test_x)))
print('test-r2:',r2_score(test_y,rm_pipe.predict(test_x)))

start_time1 = time.time()
rm_pipe.predict(test_x)
end_time1 = time.time()

print('test-time:',end_time1 - start_time1)

training-time: 0.5744781494140625
train-MSE: 0.310177082511079
train-r2: 0.2063045517655263
test-MSE: 0.3208855617415844
test-r2: 0.1842240218242862
test-time: 0.035617828369140625


2. KNN

In [32]:
from sklearn.neighbors import KNeighborsRegressor
knn_pipe = Pipeline([
    ('columns', ColumnTransformer([
        ('AADT',StandardScaler(),['AADT_ALL_VEHIC']),
        ('Crack',StandardScaler(),['MEPDG_TRANS_CRACK_LENGTH_AC'])
    ])),
    ('model', KNeighborsRegressor(n_neighbors=5))
])
start_time = time.time()
knn_pipe.fit(train_x,train_y)
end_time = time.time()
print('training-time:',end_time - start_time)
print('train-MSE:',mean_squared_error(train_y,knn_pipe.predict(train_x)))
print('train-r2:',r2_score(train_y,knn_pipe.predict(train_x)))
print('test-MSE:',mean_squared_error(test_y,knn_pipe.predict(test_x)))
print('test-r2:',r2_score(test_y,knn_pipe.predict(test_x)))

start_time1 = time.time()
knn_pipe.predict(test_x)
end_time1 = time.time()

print('test-time:',end_time1 - start_time1)

training-time: 0.05282115936279297
train-MSE: 0.1851097886904704
train-r2: 0.5263325210300673
test-MSE: 0.21784528288039667
test-r2: 0.44617966676906295
test-time: 0.05258798599243164
